In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [3]:
# import pickle
from my_util import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef, precision_recall_fscore_support, classification_report, auc

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler


import numpy as np
from scipy.optimize import differential_evolution
import pandas as pd
import time, pickle, math, warnings, os
from collections import Counter
warnings.filterwarnings('ignore')



projects = ['openstack','qt']
sampling_methods = 'DE_SMOTE_min_df_3'

remove_python_common_tokens = True

create_path_if_not_exist('./data/')
create_path_if_not_exist('./final_model/')

In [4]:
def get_combined_df(code_commit, commit_id, label, metrics_df, count_vect):
    code_df = pd.DataFrame()
    code_df['commit_id'] = commit_id
    code_df['code'] = code_commit
    code_df['label'] = label

    code_df = code_df.sort_values(by='commit_id')

    metrics_df = metrics_df.sort_values(by='commit_id')
    metrics_df = metrics_df.drop('commit_id',axis=1)

    code_change_arr = count_vect.transform(code_df['code']).astype(np.int16).toarray()
    metrics_df_arr = metrics_df.to_numpy(dtype=np.float32)

    final_features = np.concatenate((code_change_arr,metrics_df_arr),axis=1)

    return final_features, list(code_df['commit_id']), list(code_df['label'])


In [5]:
def objective_func(k, train_feature, train_label, valid_feature, valid_label):

    #smote = SMOTE(random_state=42, k_neighbors= int(np.round(k)), n_jobs=32)
    #train_feature_res, train_label_res = smote.fit_resample(train_feature, train_label)

    # Initialisiere RandomUnderSampler
    under_sampler = RandomUnderSampler( random_state=None)
    train_feature_res, train_label_res = under_sampler.fit_resample(train_feature, train_label)
    #print('Nach RandomUndersampling:', Counter(train_label_res))


    #RandomOver
    #ros = RandomOverSampler(random_state=42)
    #train_feature_res, train_label_res = ros.fit_resample(train_feature, train_label)
    #print('Nach RandomOversampling:', Counter(train_label_res))

    clf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
    clf.fit(train_feature_res, train_label_res)

    prob = clf.predict_proba(valid_feature)[:,1]
    auc = roc_auc_score(valid_label, prob)

    return -auc

## The code below this cell is used to

1. obtain the best k_neighbor of SMOTE (the value is rounded to int)
2. resample train data using SMOTE with the best k_neighbor value
3. train RF model and obtain prediction result from the model


In [6]:
def run_experiment(cur_proj):
    data_path = './data/'
    model_path = './final_model/'

    train_code, train_commit, train_label = prepare_data(cur_proj, mode='train',
                                                                  remove_python_common_tokens=remove_python_common_tokens)
    test_code, test_commit, test_label = prepare_data(cur_proj, mode='test',
                                                              remove_python_common_tokens=remove_python_common_tokens)

    commit_metrics = load_change_metrics_df(cur_proj)
    train_commit_metrics = commit_metrics[commit_metrics['commit_id'].isin(train_commit)]
    test_commit_metrics = commit_metrics[commit_metrics['commit_id'].isin(test_commit)]

    count_vect = CountVectorizer(min_df=3, ngram_range=(1,1))
    count_vect.fit(train_code)

    train_feature, train_commit_id, new_train_label = get_combined_df(train_code, train_commit, train_label, train_commit_metrics,count_vect)
    test_feature, test_commit_id, new_test_label = get_combined_df(test_code, test_commit, test_label, test_commit_metrics,count_vect)

    percent_80 = int(len(new_train_label)*0.8)

    final_train_feature = train_feature[:percent_80]
    final_train_commit_id = train_commit_id[:percent_80]
    final_new_train_label = new_train_label[:percent_80]

    valid_feature = train_feature[percent_80:]
    valid_commit_id = train_commit_id[percent_80:]
    valid_label = new_train_label[percent_80:]

    print('load data of',cur_proj, 'finish')

    bounds = [(1,20)]
    #result = differential_evolution(objective_func, bounds, args=(final_train_feature, final_new_train_label,
    #                                                              valid_feature, valid_label),
    #                               popsize=10, mutation=0.7, recombination=0.3,seed=0)

    result= result =9.40494412 #13.70396985 #QT  # result wird nicht verwendet


    print('Original dataset shape from final_new_train_label: %s' % Counter(final_new_train_label))
    #print('Original dataset shape final_train_feature %s' % Counter(final_train_feature))
    #print('\n Jetzt kommt Oversampling\n')

    #smote = SMOTE(random_state=42, n_jobs=32, k_neighbors=int(np.round(result.x)))
    #train_feature_res, train_label_res = smote.fit_resample(final_train_feature, final_new_train_label)

    under_sampler = RandomUnderSampler(random_state=None)
    train_feature_res, train_label_res = under_sampler.fit_resample(final_train_feature, final_new_train_label)
    print('Nach RandomUndersampling:', Counter(train_label_res))

    #ros = RandomOverSampler(random_state=42)
    #train_feature_res, train_label_res = ros.fit_resample(final_train_feature, final_new_train_label)
    #print('Nach RandomOversampling:', Counter(train_label_res))


    clf = RandomForestClassifier(n_estimators=300, random_state=None, n_jobs=-1)
    clf_name = 'RF'
    trained_clf, pred_df = train_eval_model(clf, train_feature_res, train_label_res,
                                       test_feature, new_test_label)
    pred_df['test_commit'] = test_commit_id
    pred_df.to_csv(data_path+cur_proj+'_'+clf_name+'_'+sampling_methods+'_prediction_result.csv')

    model_path = model_path+cur_proj+'_'+clf_name+'_'+sampling_methods+'.pkl'
    pickle.dump(trained_clf, open(model_path, 'wb'))

    print('finished',cur_proj)
    print('-'*100)

    k_of_smote = result
    best_AUC_of_obj_func = result

    return k_of_smote, best_AUC_of_obj_func

In [ ]:
%%time
openstack_k_of_smote, openstack_best_AUC_of_obj_func = run_experiment('openstack')

In [ ]:
print('The best k_neighbors of Openstack:', openstack_k_of_smote)

The best k_neighbors of Openstack: [4.2588598]


In [11]:
%%time
qt_k_of_smote, qt_best_AUC_of_obj_func = run_experiment('qt')

load data of qt finish
Original dataset shape from final_new_train_label: Counter({0: 17194, 1: 1312})
Nach RandomUndersampling: Counter({0: 1312, 1: 1312})
finished qt
----------------------------------------------------------------------------------------------------
CPU times: user 7min 39s, sys: 13.8 s, total: 7min 53s
Wall time: 6min 22s


In [ ]:
print('The best k_neighbors of Qt:', qt_k_of_smote)

The best k_neighbors of Qt: [9.36085633]


## RQ1-RQ2 result

In [8]:
RF_data_dir = './data/'

def get_recall_at_k_percent_effort(percent_effort, result_df_arg, real_buggy_commits):
    cum_LOC_k_percent = (percent_effort/100)*result_df_arg.iloc[-1]['cum_LOC']
    buggy_line_k_percent =  result_df_arg[result_df_arg['cum_LOC'] <= cum_LOC_k_percent]
    buggy_commit = buggy_line_k_percent[buggy_line_k_percent['label']==1]
    recall_k_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))

    return recall_k_percent_effort

def eval_metrics(result_df):

    pred = result_df['defective_commit_pred']
    y_test = result_df['label']

    prec, rec, f1, _ = precision_recall_fscore_support(y_test,pred,average='binary') # at threshold = 0.5
    tn, fp, fn, tp   = confusion_matrix(y_test, pred, labels=[0, 1]).ravel()
#     rec = tp/(tp+fn)

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    print('Accuracy:', accuracy)

    FAR = fp/(fp+tn) # false alarm rate
    dist_heaven = math.sqrt((pow(1-rec,2)+pow(0-FAR,2))/2.0) # distance to heaven

    AUC = roc_auc_score(y_test, result_df['defective_commit_prob'])
    print('AUC: ', AUC)
    print('FAR: ', FAR)
    print('dist_heaven: ', dist_heaven)
    print('prec:', prec)
    print('rec', rec)
    print('f1', f1)



    conf_matrix = confusion_matrix(y_test, pred)
    per_clas_acc = conf_matrix.diagonal()/conf_matrix.sum(axis=1)
    print(' per-class accuracy score: ', per_clas_acc)


    result_df['defect_density'] = result_df['defective_commit_prob']/result_df['LOC'] # predicted defect density
    result_df['actual_defect_density'] = result_df['label']/result_df['LOC'] #defect density

    result_df = result_df.sort_values(by='defect_density',ascending=False)
    actual_result_df = result_df.sort_values(by='actual_defect_density',ascending=False)
    actual_worst_result_df = result_df.sort_values(by='actual_defect_density',ascending=True)

    result_df['cum_LOC'] = result_df['LOC'].cumsum()
    actual_result_df['cum_LOC'] = actual_result_df['LOC'].cumsum()
    actual_worst_result_df['cum_LOC'] = actual_worst_result_df['LOC'].cumsum()

    real_buggy_commits = result_df[result_df['label'] == 1]

    label_list = list(result_df['label'])

    all_rows = len(label_list)

    # find Recall@20%Effort
    cum_LOC_20_percent = 0.2*result_df.iloc[-1]['cum_LOC']
    buggy_line_20_percent = result_df[result_df['cum_LOC'] <= cum_LOC_20_percent]
    buggy_commit = buggy_line_20_percent[buggy_line_20_percent['label']==1]
    recall_20_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))
    print('PCI@20: ', recall_20_percent_effort)

    # find Effort@20%Recall
    buggy_20_percent = real_buggy_commits.head(math.ceil(0.2 * len(real_buggy_commits)))
    buggy_20_percent_LOC = buggy_20_percent.iloc[-1]['cum_LOC']
    effort_at_20_percent_LOC_recall = int(buggy_20_percent_LOC) / float(result_df.iloc[-1]['cum_LOC'])
    print('effort@20: ', effort_at_20_percent_LOC_recall)

    # find P_opt
    percent_effort_list = []
    predicted_recall_at_percent_effort_list = []
    actual_recall_at_percent_effort_list = []
    actual_worst_recall_at_percent_effort_list = []

    for percent_effort in np.arange(10,101,10):
        predicted_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, result_df, real_buggy_commits)
        actual_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_result_df, real_buggy_commits)
        actual_worst_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_worst_result_df, real_buggy_commits)

        percent_effort_list.append(percent_effort/100)

        predicted_recall_at_percent_effort_list.append(predicted_recall_k_percent_effort)
        actual_recall_at_percent_effort_list.append(actual_recall_k_percent_effort)
        actual_worst_recall_at_percent_effort_list.append(actual_worst_recall_k_percent_effort)

    p_opt = 1 - ((auc(percent_effort_list, actual_recall_at_percent_effort_list) -
                 auc(percent_effort_list, predicted_recall_at_percent_effort_list)) /
                (auc(percent_effort_list, actual_recall_at_percent_effort_list) -
                auc(percent_effort_list, actual_worst_recall_at_percent_effort_list)))

    print('p_opt: ', p_opt)

    return prec, rec, f1, AUC, FAR, dist_heaven, recall_20_percent_effort, effort_at_20_percent_LOC_recall, p_opt, per_clas_acc

def eval_result(proj_name,sampling_method = 'DE_SMOTE_min_df_3'):

    RF_result = pd.read_csv(RF_data_dir+proj_name+'_RF_'+sampling_method+'_prediction_result.csv')

    RF_result.columns = ['Unnamed', 'defective_commit_prob','defective_commit_pred','label','test_commit'] # for new result

    test_code, test_commit, test_label = prepare_data(proj_name, mode='test',
                                                              remove_python_common_tokens=remove_python_common_tokens)

    # get LOC of each commit
    RF_LOC = [len(code.splitlines()) for code in test_code]
    RF_df = pd.DataFrame()
    RF_df['commit_id'] = test_commit
    RF_df['LOC'] = RF_LOC

    RF_result = pd.merge(RF_df, RF_result,how='inner',left_on = 'commit_id', right_on='test_commit')
    prec, rec, f1, auc, FAR, dist_heaven, recall_20_percent_effort, effort_at_20_percent_LOC_recall,p_opt, per_clas_acc = eval_metrics(RF_result)


    print('Precision: {:.2f}, Recall: {:.2f}, F1: {:.2f}, AUC: {:.2f}, FAR: {:.2f}, d2h: {:.2f}, PCI@20%LOC: {:.2f}, Effort@20%Recall: {:.2f}, POpt: {:.2f}'.format(prec, rec, f1, auc, FAR, dist_heaven, recall_20_percent_effort, effort_at_20_percent_LOC_recall,p_opt))
    #print('Per-class accuracy score:', per_clas_acc[1] )

In [ ]:
eval_result('openstack')

In [12]:
eval_result('qt')

Accuracy: 0.7950213924542979
AUC:  0.804512086845887
FAR:  0.19095477386934673
dist_heaven:  0.3057701460187278
prec: 0.19718309859154928
rec 0.6120218579234973
f1 0.2982689747003995
 per-class accuracy score:  [0.80904523 0.61202186]
PCI@20:  0.6502732240437158
effort@20:  0.026303846116706418
p_opt:  0.8783652286733699
Precision: 0.20, Recall: 0.61, F1: 0.30, AUC: 0.80, FAR: 0.19, d2h: 0.31, PCI@20%LOC: 0.65, Effort@20%Recall: 0.03, POpt: 0.88


## RQ3 result

note: the_best_k_neighbors is obtained from model training phase

In [ ]:
def check_train_time(cur_proj, the_best_k_neighbors):
    data_path = './data/'
    model_path = './final_model/'

    train_code, train_commit, train_label = prepare_data(cur_proj, mode='train',
                                                                  remove_python_common_tokens=remove_python_common_tokens)

    commit_metrics = load_change_metrics_df(cur_proj)
    train_commit_metrics = commit_metrics[commit_metrics['commit_id'].isin(train_commit)]

    count_vect = CountVectorizer(min_df=3, ngram_range=(1,1))
    count_vect.fit(train_code)

    print('fit countvectorizer finished')

    train_feature, train_commit_id, new_train_label = get_combined_df(train_code, train_commit, train_label, train_commit_metrics,count_vect)

    percent_80 = int(len(new_train_label)*0.8)

    final_train_feature = train_feature[:percent_80]
    final_train_commit_id = train_commit_id[:percent_80]
    final_new_train_label = new_train_label[:percent_80]

    smote = SMOTE(random_state=42, n_jobs=1, k_neighbors=the_best_k_neighbors)

    train_feature_res, train_label_res = smote.fit_resample(final_train_feature, final_new_train_label)

    clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    clf_name = 'RF'

    start = time.time()

    clf.fit(train_feature_res, train_label_res)

    end = time.time()

    train_time = end-start
    print('train time of {} is {:.3f} secs'.format(cur_proj,train_time))

In [ ]:
%%time
check_train_time('openstack', 9)

fit countvectorizer finished
train time of openstack is 10.795 secs
CPU times: user 1min 53s, sys: 4.2 s, total: 1min 57s
Wall time: 46 s


In [ ]:
%%time
check_train_time('qt', 14)

fit countvectorizer finished
train time of qt is 106.038 secs
CPU times: user 16min 11s, sys: 37.6 s, total: 16min 49s
Wall time: 5min 41s
